# 04. Optimizers

In [1]:
import torch

from tqdm import tqdm
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import v2
import torchvision.transforms as transforms
import torch.nn.init as init

import numpy as np

from multiprocessing import freeze_support
import time
from timed_decorator.simple_timed import timed


In [2]:
BATCH_SIZE = 64

In [3]:
class MnistDataset(Dataset):
    def __init__(self, in_transform, out_transform, num_class=10, num_rand_class=4, freq_rand=10):
        dataset  = datasets.MNIST(root='./data', train=True, transform=None, download=True)
        self.data    = dataset.data.numpy()
        self.targets = dataset.targets.numpy()
        self.in_transform  = in_transform
        self.out_transform = out_transform
        self.rand_in   = torch.empty(self.data.shape[1:])
        self.num_class = num_class
        self.num_rand_class = num_rand_class+self.num_class
        self.freq_rand = freq_rand
        self.count     = 0

    def getNumClass(self):
        return self.num_rand_class
    
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, i: int):
        if (self.count > self.freq_rand):
            inputs  = self.rand_in.uniform_(-1.0, 1.0).numpy()
            outputs = torch.randint(low=self.num_class, high=self.num_rand_class, size=(1,)).item()
            self.count = 0
        else:
            inputs  = self.data[i]
            outputs = self.targets[i]
        self.count += 1
        if (self.in_transform is not None):
            inputs = self.in_transform(inputs)
        if (self.out_transform is not None):
            outputs = self.out_transform(outputs)
        return (inputs, outputs)

class MnistDatasetVal(Dataset):
    def __init__(self, in_transform, out_transform, num_class=10, num_rand_class=4):
        dataset  = datasets.MNIST(root='./data', train=False, transform=None, download=True)
        self.data    = dataset.data.numpy()
        self.targets = dataset.targets.numpy()
        self.in_transform  = in_transform
        self.out_transform = out_transform
        self.num_class = num_class
        self.num_rand_class = num_rand_class+self.num_class

    def getNumClass(self):
        return self.num_rand_class
    
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, i: int):
        inputs  = self.data[i]
        outputs = self.targets[i]
        if (self.in_transform is not None):
            inputs = self.in_transform(inputs)
        if (self.out_transform is not None):
            outputs = self.out_transform(outputs)
        return (inputs, outputs)

In [4]:

val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    torch.flatten,
])

@timed(use_seconds=True, show_args=True, return_time=True)
def load_data(num_workers: int):
    dataset = MnistDataset(in_transform=val_transforms, out_transform=None, num_class=10, num_rand_class=10, freq_rand=5)
    dataloader = DataLoader(dataset, batch_size=1, num_workers=num_workers)
    for _ in dataloader:
        pass  # Simulate training

In [5]:

freeze_support()
_, t0 = load_data(0)
_, t1 = load_data(2)
_, t2 = load_data(4)
_, t3 = load_data(8)
print()
print(f"num_workers: {0}, time: {t0} seconds")
print(f"num_workers: {2}, time: {t1} seconds")
print(f"num_workers: {4}, time: {t2} seconds")
print(f"num_workers: {8}, time: {t3} seconds")
print(f"Speedup: {t0/t1}, {t0/t2}, {t0/t3}")

KeyboardInterrupt: 

In [6]:

num_workers = 0
train_set = MnistDataset(in_transform=val_transforms, out_transform=None, num_class=10, num_rand_class=10, freq_rand=5)
val_set   = MnistDatasetVal(in_transform=val_transforms, out_transform=None, num_class=10, num_rand_class=5)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, drop_last=False)
val_loader   = DataLoader(val_set,   batch_size=64, shuffle=False, num_workers=num_workers)

In [ ]:

def test_ds_fn(dataset, i):
    inputs, outputs = dataset[i]
    print("inputs {}, outputs {}, type {}".format(inputs.shape, outputs, type(outputs)))

In [ ]:
for i in range(len(train_set)):
    test_ds_fn(train_set, i)

In [7]:

class Trainer:
    def __init__(
            self,
            model: nn.Module,
            optimizer: torch.optim.Optimizer,
            criterion: nn.Module,
            use_cpu=False
    ):
        self.device = torch.device("cpu") if use_cpu else torch.accelerator.current_accelerator()
        print(f"Using device: {self.device}")

        # We don't need to shuffle the validation set
        self.model = model.to(self.device)  # The model must be on the same device
        self.criterion = criterion.to(self.device)  # Required for some loss functions
        self.optimizer = optimizer

    def train(self, train_ds):
        self.model.train()

        total = 0
        correct = 0
        total_loss = 0

        for data, target in tqdm(train_ds, desc="Training", leave=False, disable=True):  # Disable on notebook
            # We must move the data to the same device as the model
            data   = data.to(self.device)
            target = target.to(self.device)
            # We can also use non_blocking=True to speed up the transfer for large tensors
            # data = data.to(self.device, non_blocking=True)
            # but this is useful only for pinned memory transfers (CPU-to-GPU)
            # In most cases, the improvement is negligible

            predicted = self.model(data)
            loss = self.criterion(predicted, target)
            loss.backward()

            self.optimizer.step()
            self.optimizer.zero_grad()

            correct += (predicted.argmax(dim=1) == target).sum().item()
            total   += data.size(0)
            total_loss += loss.item() * data.size(0)

        return total_loss / total, correct / total

    # @torch.no_grad()  # This is what you usually see in tutorials
    @torch.inference_mode()  # This is the recommended way to do this
    def val(self, val_ds):
        self.model.eval()

        total = 0
        correct = 0
        total_loss = 0

        for data, target in tqdm(val_ds, desc="Validation", leave=False, disable=True):  # Disable on notebook
            data = data.to(self.device)
            target = target.to(self.device)

            predicted = self.model(data)
            loss = self.criterion(predicted, target)

            correct += (predicted.argmax(dim=1) == target).sum().item()
            total   += data.size(0)
            total_loss += loss.item() * data.size(0)

        return total_loss / total, correct / total

    def run(self, train_ds, val_ds, epochs: int):
        print(f"Running {epochs} epochs")
        with tqdm(range(epochs), desc="Training") as pbar:
            for _ in pbar:
                tr_loss, tr_acc = self.train(train_ds)
                va_loss, va_acc = self.val(val_ds)
                pbar.set_postfix(train_loss=tr_loss, train_acc=tr_acc, val_loss=va_loss, val_acc=va_acc)
        print("Last validation accuracy: ", va_acc)
        print()

In [8]:

# PyTorch models inherit from torch.nn.Module
class MnistClassifier(nn.Module):
    def __init__(self, initizlizer=None):
        super(MnistClassifier, self).__init__()
        self.layers = [nn.Linear(28 * 28, 20),
                    nn.ReLU(inplace=True),
                    nn.Linear(20, train_set.getNumClass()),]
        self.block = nn.Sequential(*self.layers)
        self.initizlizer = initizlizer 

    def reset_parameters(self):
        if (self.initizlizer is not None):
            self.initizlizer(self.block.parameters())
        else:
            for layer in self.layers:
                if ((hasattr(layer, "reset_parameters")) and (callable(getattr(layer, "reset_parameters")))):
                    layer.reset_parameters()

    def forward(self, x):
        x = self.block(x)
        return x


In [12]:

def main(model, train_ds, val_ds, epochs: int, learning_rate:float, optimizer: str, lr_hyperparameters=None):
    print(f"Running {epochs} epochs with {optimizer} optimizer")
    model.reset_parameters()
    if (lr_hyperparameters is not None):
        lr = lr_hyperparameters.pop("lr", None)
        if  ((lr is None) and (isinstance(learning_rate, float))):
            lr = learning_rate
        else:
            lr = 0.001
    else:
        lr = learning_rate
    print("Rata de antrenare {}".format(lr))
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    elif optimizer == "sgd_momentum":
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif optimizer == "sgd_momentum_nesterov":
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True)
    elif optimizer == "sgd_momentum_nesterov_weight_decay":
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True, weight_decay=0.001)
    elif optimizer == "adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    elif optimizer == "adamw":
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    elif optimizer == "rmsprop":
        optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
    else:
        raise NotImplementedError(f"Optimizer {optimizer} not implemented")

    if ((not isinstance(learning_rate, float)) and (callable(learning_rate))):
        learning_rate(optimizer, **lr_hyperparameters)
    trainer = Trainer(model, optimizer, nn.CrossEntropyLoss())
    trainer.run(train_ds, val_ds, epochs)

Recommended resources:
* https://emiliendupont.github.io/2018/01/24/optimization-visualization/
* The official documentation for each optimizer

In [10]:

lst_optimizers = ["sgd", "sgd_momentum", "sgd_momentum_nesterov", "sgd_momentum_nesterov_weight_decay", "adam", "adamw", "rmsprop"]

In [98]:

epoch = 30
learning_rate = 0.001
model = MnistClassifier(init_xavier)
print("Model cu initializare xavier, rata de antrenare {}".format(learning_rate))
for str_optimizer in lst_optimizers:
    main(model, train_loader, val_loader, epoch, learning_rate, str_optimizer)
learning_rate = 0.0001
print("\n\n\nModel cu initializare xavier, rata de antrenare {}".format(learning_rate))
for str_optimizer in lst_optimizers:
    main(model, train_loader, val_loader, epoch, learning_rate, str_optimizer)
learning_rate = 0.01
print("\n\n\nModel cu initializare xavier, rata de antrenare {}".format(learning_rate))
for str_optimizer in lst_optimizers:
    main(model, train_loader, val_loader, epoch, learning_rate, str_optimizer)

# Engineering: Why do you think the training is so slow? Can you make it faster?
# Science: Why do you think the results are not better? What can we do to improve them?

Model cu initializare xavier, rata de antrenare 0.001
Running 30 epochs with sgd optimizer
Using device: cuda
Running 30 epochs


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:30<00:00,  5.03s/it, train_acc=0.769, train_loss=0.693, val_acc=0.906, val_loss=0.333]


Last validation accuracy:  0.9065

Running 30 epochs with sgd_momentum optimizer
Using device: cuda
Running 30 epochs


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:30<00:00,  5.02s/it, train_acc=0.795, train_loss=0.584, val_acc=0.928, val_loss=0.246]


Last validation accuracy:  0.9282

Running 30 epochs with sgd_momentum_nesterov optimizer
Using device: cuda
Running 30 epochs


...

Last validation accuracy:  0.2669

Running 30 epochs with adamw optimizer
Using device: cuda
Running 30 epochs


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:33<00:00,  5.11s/it, train_acc=0.726, train_loss=0.822, val_acc=0.863, val_loss=0.487]


Last validation accuracy:  0.8629

Running 30 epochs with rmsprop optimizer
Using device: cuda
Running 30 epochs


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:30<00:00,  5.03s/it, train_acc=0.729, train_loss=0.842, val_acc=0.777, val_loss=0.781]

Last validation accuracy:  0.7772



In [99]:

epoch = 30
model = MnistClassifier()
learning_rate = 0.001
print("Model cu initializare Kaiming Uniform, rata de antrenare {}".format(learning_rate))
for str_optimizer in lst_optimizers:
    main(model, train_loader, val_loader, epoch, learning_rate, str_optimizer)

learning_rate = 0.0001
print("Model cu initializare Kaiming Uniform, rata de antrenare {}".format(learning_rate))
for str_optimizer in lst_optimizers:
    main(model, train_loader, val_loader, epoch, learning_rate, str_optimizer)
    
learning_rate = 0.01
print("Model cu initializare Kaiming Uniform, rata de antrenare {}".format(learning_rate))
for str_optimizer in lst_optimizers:
    main(model, train_loader, val_loader, epoch, learning_rate, str_optimizer)
# Engineering: Why do you think the training is so slow? Can you make it faster?
# Science: Why do you think the results are not better? What can we do to improve them?

Model cu initializare Kaiming Uniform, rata de antrenare 0.001
Running 30 epochs with sgd optimizer
Using device: cuda
Running 30 epochs


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:27<00:00,  4.93s/it, train_acc=0.772, train_loss=0.674, val_acc=0.909, val_loss=0.323]


Last validation accuracy:  0.9087

Running 30 epochs with sgd_momentum optimizer
Using device: cuda
Running 30 epochs


Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:30<00:00,  5.01s/it, train_acc=0.803, train_loss=0.55, val_acc=0.94, val_loss=0.209]


Last validation accuracy:  0.9396

Running 30 epochs with sgd_momentum_nesterov optimizer
Using device: cuda
Running 30 epochs


...

Last validation accuracy:  0.8142

Running 30 epochs with adamw optimizer
Using device: cuda
Running 30 epochs


Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:33<00:00,  5.10s/it, train_acc=0.45, train_loss=1.52, val_acc=0.526, val_loss=1.28]


Last validation accuracy:  0.5265

Running 30 epochs with rmsprop optimizer
Using device: cuda
Running 30 epochs


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:32<00:00,  5.09s/it, train_acc=0.675, train_loss=1.01, val_acc=0.79, val_loss=0.692]

Last validation accuracy:  0.79



In [14]:

epoch = 30
model = MnistClassifier()
learning_rate = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts
lr_hyperparameters = dict(lr=0.01, T_0=10, T_mult=1, eta_min=0.001, last_epoch=-1)
print("Model cu initializare Kaiming Uniform, lr_scheduler {}".format("CosineAnnealingWarmRestarts"))
for str_optimizer in lst_optimizers:
    main(model, train_loader, val_loader, epoch, learning_rate, str_optimizer, lr_hyperparameters)

lr_hyperparameters = dict(lr=0.01, T_0=10, T_mult=2, eta_min=0.01, last_epoch=-1)
print("Model cu initializare Kaiming Uniform, lr_scheduler {}".format("CosineAnnealingWarmRestarts"))
for str_optimizer in lst_optimizers:
    main(model, train_loader, val_loader, epoch, learning_rate, str_optimizer, lr_hyperparameters)

learning_rate = torch.optim.lr_scheduler.CosineAnnealingLR
lr_hyperparameters = dict(lr=0.01, T_max=10, eta_min=0)
print("Model cu initializare Kaiming Uniform, lr_scheduler {}".format("CosineAnnealingLR"))
for str_optimizer in lst_optimizers:
    main(model, train_loader, val_loader, epoch, learning_rate, str_optimizer, lr_hyperparameters)

learning_rate = torch.optim.lr_scheduler.OneCycleLR
lr_hyperparameters = dict(lr=0.01, max_lr=0.1, steps_per_epoch=10, epochs=epoch)
print("Model cu initializare Kaiming Uniform, lr_scheduler {}".format("OneCycleLR"))
for str_optimizer in lst_optimizers:
    main(model, train_loader, val_loader, epoch, learning_rate, str_optimizer, lr_hyperparameters)

lr_hyperparameters = dict(lr=0.01, max_lr=0.1, steps_per_epoch=10, epochs=epoch, anneal_strategy='linear')
print("Model cu initializare Kaiming Uniform, lr_scheduler {}".format("OneCycleLR, linear"))
for str_optimizer in lst_optimizers:
    main(model, train_loader, val_loader, epoch, learning_rate, str_optimizer, lr_hyperparameters)

learning_rate = torch.optim.lr_scheduler.LambdaLR
lambda1 = lambda epoch: 1.05**epoch
lr_hyperparameters = dict(lr=0.01, lr_lambda=lambda1)
print("Model cu initializare Kaiming Uniform, lr_scheduler {}".format("LambdaLR"))
for str_optimizer in lst_optimizers:
    main(model, train_loader, val_loader, epoch, learning_rate, str_optimizer, lr_hyperparameters)

    
# Engineering: Why do you think the training is so slow? Can you make it faster?
# Science: Why do you think the results are not better? What can we do to improve them?

Model cu initializare Kaiming Uniform, lr_scheduler CosineAnnealingWarmRestarts
Running 30 epochs with sgd optimizer
Rata de antrenare 0.001
Using device: cuda
Running 30 epochs


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:24<00:00,  4.80s/it, train_acc=0.771, train_loss=0.678, val_acc=0.911, val_loss=0.322]


Last validation accuracy:  0.9106

Running 30 epochs with sgd_momentum optimizer
Rata de antrenare 0.001
Using device: cuda
Running 30 epochs


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:25<00:00,  4.86s/it, train_acc=0.807, train_loss=0.542, val_acc=0.943, val_loss=0.192]


Last validation accuracy:  0.943

Running 30 epochs with sgd_momentum_nesterov optimizer
Rata de antrenare 0.001
Using device: cuda
Running 30 epochs


...

Last validation accuracy:  0.9426

Running 30 epochs with adamw optimizer
Rata de antrenare 0.001
Using device: cuda
Running 30 epochs


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:31<00:00,  5.06s/it, train_acc=0.777, train_loss=0.652, val_acc=0.911, val_loss=0.322]


Last validation accuracy:  0.9112

Running 30 epochs with rmsprop optimizer
Rata de antrenare 0.001
Using device: cuda
Running 30 epochs


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [02:28<00:00,  4.96s/it, train_acc=0.767, train_loss=0.685, val_acc=0.891, val_loss=0.383]

Last validation accuracy:  0.8914



# convolution

In [ ]:

def init_xavier(parameters):
    for l_params in parameters:
        if (l_params is not None):
            if (l_params.dim() > 1):
                init.xavier_uniform_(l_params)
            else:
                init.zeros_(l_params)

# PyTorch models inherit from torch.nn.Module
class MnistClassifier(nn.Module):
    def __init__(self, initizlizer=None):
        super(MnistClassifier, self).__init__()
        self.layers = [nn.Linear(28 * 28, 20),
                    nn.ReLU(inplace=True),
                    nn.Linear(20, train_set.getNumClass()),]
        self.block = nn.Sequential(*self.layers)
        self.initizlizer = initizlizer 

    def reset_parameters(self):
        if (self.initizlizer is not None):
            self.initizlizer(self.block.parameters())
        else:
            for layer in self.layers:
                if ((hasattr(layer, "reset_parameters")) and (callable(getattr(layer, "reset_parameters")))):
                    layer.reset_parameters()

    def forward(self, x):
        x = self.block(x)
        return x


## Excercises:

You may start with excercise 3 if 1 and 2 prove to be too difficult. Implementing your own pipeline might bring you closer to the solution.

1. Modify this pipeline in order to improve the training speed. Test various hypotheses. Starting with a simple pipeline allows you to make more focused progress in understanding the bottlenecks. Measuring the is the most important part of the process. Python multi-threading/multi-processing is not the right answer.
2. Focus on increasing the accuracy of the model. You should be able to easily get 95% accuracy on the validation set.
3. Implement your own pipeline from scratch. Implementing it yourself provides a better understanding. Do not be shackeled by this model.
4. Try to get 98% accuracy on the validation set with all optimizers, using this pipeline. It is not very complicated.
5. Try to get 98% with your pipeline.

> !!! **Do not make training decisions based on the validation data! Otherwise you will risk overfitting!**

---


| All     | [beginner_pytorch/](https://github.com/Tensor-Reloaded/AI-Learning-Hub/blob/main/resources/beginner_pytorch) |
|---------|-- |
| Prev    | [Simple Training](https://github.com/Tensor-Reloaded/AI-Learning-Hub/blob/main/resources/beginner_pytorch/03_simple_training.ipynb) |
| Current | [Optimizers](https://github.com/Tensor-Reloaded/AI-Learning-Hub/blob/main/resources/beginner_pytorch/04_optimizers.ipynb) |
| Next    | [LR Schedulers](https://github.com/Tensor-Reloaded/AI-Learning-Hub/blob/main/resources/beginner_pytorch/05_lr_schedulers.ipynb) |